In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# CREDENTIALS & CONFIG
API_URL = os.getenv("API_URL")
API_KEY = os.getenv("API_KEY")
DATA_DIR = "../data/raw"

# Check if data directory exists
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    print(f"[+] Created directory: {DATA_DIR}")

# LIST OF ALL KNOWN ACTION TYPES IN THE DATABASE
ACTION_TYPES = [
    "RAID_INTERACTION",
    "BATTLE_END",
    "POKEMON_CAPTURED",
    "POKEMON_RELEASED",
    "POKEMON_HATCHED",
    "GTS_TRANSACTION",
    "SESSION_LOGIN",
    "SESSION_LOGOUT",
    "SESSION_SNAPSHOT",
    "COMMAND_USAGE",
    "PLAYER_DEATH"
]

print(f"[+] Configuration loaded. Target API: {API_URL}")

In [ ]:
def download_and_save(action_type):
    print(f"[+] Requesting data for: {action_type}...")

    try:
        response = requests.get(
            API_URL,
            headers={"Authorization": f"Bearer {API_KEY}"},
            params={"action": action_type}
        )

        if response.status_code == 200:
            data = response.json()

            if not data:
                print(f"[-] No data found for {action_type}. Skipping.")
                return

            # Convert to DataFrame
            df = pd.DataFrame(data)

            # Construct filename: data/dataset_BATTLE_END_raw.csv
            filename = f"{DATA_DIR}/dataset_{action_type}_raw.csv"

            # Save Raw Data
            df.to_csv(filename, index=False)
            print(f"[+] Successfully saved {len(df)} rows to '{filename}'")

        else:
            print(f"[-] API Error {response.status_code}: {response.text}")

    except Exception as e:
        print(f"[-] Critical Error downloading {action_type}: {e}")

In [ ]:
# ---------------------------------------------------------
# EXECUTE BATCH DOWNLOAD
# ---------------------------------------------------------
print("[INFO] Starting Batch Download...")

for action in ACTION_TYPES:
    download_and_save(action)

print("[INFO] Batch Download Complete.")